<a href="https://colab.research.google.com/github/alexwz/ComboNER/blob/master/TF20Combo_NER_final_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://nbviewer.jupyter.org/github/zhedongzheng/finch/blob/master/finch/tensorflow2/spoken_language_understanding/atis/main/bigru.ipynb

In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls -lh '/content/drive/My Drive/combo/ner/onefile.conll'

-rw------- 1 root root 11M Jun 19 14:53 '/content/drive/My Drive/combo/ner/onefile.conll'


In [2]:
!pip install bpemb==0.3.3
!pip install conllu==2.2
!pip install --upgrade tensorflow_addons

     |████████████████████████████████| 1.2MB 12.0MB/s 
     |████████████████████████████████| 686kB 8.0MB/s 


https://nlp.h-its.org/bpemb/

In [19]:
from bpemb import BPEmb
from conllu import parse_incr
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import classification_report, f1_score, accuracy_score
import tensorflow as tf
import tensorflow_addons as tfa
import pprint 
import time, sys, logging
from tensorboard.plugins.hparams import api as hp

print("TensorFlow Version", tf.__version__)
print('GPU Enabled:', tf.config.list_physical_devices('GPU'))

params = {
  'train_path_ud': "/content/drive/My Drive/combo/PTCombo/UD/PL/pl_pdb-ud-train.conllu",
  'test_path_ud': "/content/drive/My Drive/combo/PTCombo/UD/PL/pl_pdb-ud-test.conllu",
  'train_path_ner': "/content/drive/My Drive/combo/ner/nkjp-ner-train.conllu",
  'test_path_ner': "/content/drive/My Drive/combo/ner/nkjp-ner-test.conllu",
  'batch_size': 32,
  'num_samples': 4978,
  'rnn_units': 100,
  'rnn_units_ner': 50,
  'dropout_rate': 0.2,
  'clip_norm': 5.0,
  'lr': 3e-4,
  'num_patience': 2,
  'pos_size': 100,
  'heads_size': 100,
  'token_len': 6,
  'sentence_len': 67,
  'embedding_size': 100,
  'vocabulary_size': 50000,
  'debug': False
}


TensorFlow Version 2.5.0
GPU Enabled: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
!nvidia-smi

Tue Jun 22 05:29:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    27W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11926682288099830810, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 16183459840
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 1136177403853422952
 physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"]

Inicjalizacja oraz uruchomienie label encoderow:

In [6]:
def get_label_encoders(bpemb_pl):
  le_upostag = preprocessing.LabelEncoder()
  le_deprel = preprocessing.LabelEncoder()

  # this method iterates CONLL files and outputs relevant fields 
  # to generate Label Encoder IDs and compute various stats
  def data_generator_tokens(f_path, params, bpemb_pl):
    #print('Reading', f_path)
    data_file = open(f_path, "r", encoding="utf-8")
    for tokenlist in parse_incr(data_file):

        orths = [ t['form'] for t in tokenlist ]
        lemmas = [ t['lemma'] for t in tokenlist ]
        upostags = [ t['upostag'] for t in tokenlist ]
        heads = [ t['head'] for t in tokenlist ]
        # use only major dependency labels -> 33 instead of 68:
        deprels = [ t['deprel'].split(':')[0] for t in tokenlist ]

        assert len(orths) == len(lemmas) == len(upostags) == len(heads) == len(deprels)

        # subword-level collections:
        sub_tokenids = []
        sub_upostags = []
        sub_heads = []
        sub_deprels = []

        for nr, orth in enumerate(orths):
          tokenids = bpemb_pl.encode_ids(orth)
          sub_tokenids.extend( tokenids )
          sub_upostags.extend( [upostags[nr]] * len(tokenids) )
          sub_heads.extend( [heads[nr]] * len(tokenids) )
          sub_deprels.extend( [deprels[nr]] * len(tokenids) )

        yield (sub_tokenids, (sub_upostags, sub_heads, sub_deprels))

  toksizes = [] ; postags_all = [] ; deprels_all = [] ; sentence_lens = []
  nr = 0
  for conll_tuple in data_generator_tokens(params["train_path_ud"], params, bpemb_pl):
    sub_tokenids, (upostags, heads, deprels) = conll_tuple[:]
    sentence_lens.append( len(sub_tokenids) )
    postags_all.extend(upostags)
    deprels_all.extend(deprels)

  le_upostag.fit(postags_all)
  le_deprel.fit(deprels_all)
  return le_upostag, le_deprel

In [7]:
tf.keras.backend.epsilon()

1e-07

In [ ]:
tf.keras.backend.set_epsilon(1e-5)

In [8]:
def is_descending(history: list):
  history = history[-(params['num_patience']+1):]
  for i in range(1, len(history)):
    if history[i-1] <= history[i]:
      return False
  return True    

In [9]:
# this method iterates CONLL files and outputs padded IDs  
# it's intended as TensorFlow graph input
def data_generator_ids_UD(f_path, params, bpemb_pl, le_upostag, le_deprel):
  print('Reading', f_path)
  data_file = open(f_path, "r", encoding="utf-8")
  for tokenlist in parse_incr(data_file):

      orths = [ t['form'] for t in tokenlist ]
      lemmas = [ t['lemma'] for t in tokenlist ]
      upostags = [ t['upostag'] for t in tokenlist ]
      heads = [ t['head'] for t in tokenlist ]
      # use only major dependency labels -> 33 instead of 68:
      deprels = [ t['deprel'].split(':')[0] for t in tokenlist ]
      
      assert len(orths) == len(lemmas) == len(upostags) == len(heads) == len(deprels)

      # fix broken data:
      heads = [ 0 if h is None else h for h in heads]
      heads = [ 0 if h >=params["sentence_len"] else h for h in heads]

      # subword-level collections:
      sub_tokenids = []
      sub_upostags = []
      sub_heads = []
      sub_deprels = []

      for nr, orth in enumerate(orths):
        tokenids = bpemb_pl.encode_ids(orth)
        sub_tokenids.extend( tokenids )
        sub_upostags.extend( [upostags[nr]] * len(tokenids) )
        sub_heads.extend( [heads[nr]] * len(tokenids) )
        sub_deprels.extend( [deprels[nr]] * len(tokenids) )
      
      sub_upostags_ids = le_upostag.transform(sub_upostags)
      sub_deprels_ids = le_deprel.transform(sub_deprels)

      ner_slot = [0] * len(heads)

      yield (sub_tokenids, (sub_upostags_ids, sub_heads, sub_deprels_ids, ner_slot ))

def dataset_UD(is_training, params, bpemb_pl, le_upostag, le_deprel):

  _shapes = ( (None,), ((None,), (None,), (None,), (None,)))
  _types = (tf.int32, (tf.int32, tf.int32, tf.int32, tf.int32))
  _pads = (0, (0, 0, 0, 0)) # no neg values here!
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator_ids_UD(params['train_path_ud'], params, bpemb_pl, le_upostag, le_deprel),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['num_samples'])
    ds = ds.padded_batch(params['batch_size'], _shapes )#, _pads) # default mask is 0
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator_ids_UD(params['test_path_ud'], params, bpemb_pl, le_upostag, le_deprel),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes )#, _pads) # default mask is 0
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [10]:
class Model(tf.keras.Model):

  def __init__(self, params: dict, embedding_matrix, bpemb_pl, le_upostag, le_deprel, le_ner):
    super().__init__()
    
    self.embeddings = tf.keras.layers.Embedding(bpemb_pl.vocab_size, bpemb_pl.vectors.shape[1],
                                                embeddings_initializer=tf.keras.initializers.Constant(bpemb_pl.vectors), 
                                                trainable=True, mask_zero=True)
    
    self.input_dropout = tf.keras.layers.Dropout(params['dropout_rate'])
    self.bidir_rnn1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        params['rnn_units'], return_sequences=True), name='rnn')

    # part-of-speech:
    self.out_linear_pos = tf.keras.layers.Dense(len(le_upostag.classes_) , name='output_pos')

    # dependency parser dependencies:
    # heads need N+1 classes, the extra last class is for None entries in CONLLU:    
    self.linear_dep_arc = tf.keras.layers.Dense(params["sentence_len"]+2, activation='tanh', name='dep_arc')
    self.linear_head_arc = tf.keras.layers.Dense(params["sentence_len"]+2, activation='tanh', name='head_arc')
    self.dropout_dep = tf.keras.layers.Dropout(params['dropout_rate'])
    self.dropout_head = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_linear_head_dep_dot = tf.keras.layers.Dot(axes=2)

    # dependency parser labels:
    self.out_linear_dep_rel = tf.keras.layers.Dense(len(le_deprel.classes_), activation='tanh' )
    self.out_linear_head_rel = tf.keras.layers.Dense(len(le_deprel.classes_), activation='tanh' )
    
    self.dropout_dep_rel = tf.keras.layers.Dropout(params['dropout_rate'])
    self.dropout_head_rel = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_linear_head_dep_rel_dot = tf.keras.layers.Dot(axes=2)
    self.lambda_reshape = tf.keras.layers.Lambda(lambda x: tf.keras.backend.permute_dimensions(x, (0, 2, 1)))
    self.dep_rel_conc = tf.keras.layers.Concatenate(axis=2)
    self.dropout_out_dep_rel = tf.keras.layers.Dropout(params['dropout_rate'])
    self.out_dense_deprel = tf.keras.layers.Dense(len(le_deprel.classes_), name='output_deprel')

    # Named Entity Recognition:
    self.conc_embeddings_rnn  = tf.keras.layers.Concatenate(axis=2)
    self.bidir_rnn_ner = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(
        params['rnn_units_ner'], return_sequences=True), name='rnn_ner')
    self.out_dense_ner = tf.keras.layers.Dense(len(le_ner.classes_) , name='output_ner')
  
  def call(self, inputs, training=False):
    if inputs.dtype != tf.int32:
      inputs = tf.cast(inputs, tf.int32)

    subwordvecs = self.embeddings(inputs)

    input_dropouted = self.input_dropout(subwordvecs, training=training)
    x = self.bidir_rnn1(input_dropouted)

    pos_pred = self.out_linear_pos(x)
    # after out_linear_pos x_pos.shape: (16, 32, 16)

    dep_arc_emb = self.dropout_dep( self.linear_dep_arc(x), training=training)
    head_arc_emb = self.dropout_head( self.linear_head_arc(x), training=training)
    heads_pred = self.out_linear_head_dep_dot([dep_arc_emb, head_arc_emb])
    # x_heads.shape: (16, 24, 24)
    
    # dependency parser labels:
    dep_rel_emb = self.dropout_dep_rel( self.out_linear_dep_rel(x), training=training)
    head_rel_emb = self.dropout_head_rel( self.out_linear_head_rel(x), training=training)
    head_emb_T = self.lambda_reshape(head_rel_emb) # reshape

    deprel_pred = self.out_linear_head_dep_rel_dot([heads_pred, head_emb_T])
    deprel_conctd = self.dep_rel_conc([deprel_pred, dep_rel_emb])
    deprel_conc_dens = self.out_dense_deprel(deprel_conctd)
    dep_rels_pred = self.dropout_out_dep_rel( deprel_conc_dens, training=training)

    # named entity recognition:
    ner_input_concatenated = self.conc_embeddings_rnn([x, subwordvecs])
    rnn_ner_output = self.bidir_rnn_ner(x)
    ner_pred = self.out_dense_ner(rnn_ner_output)
    #  concatenate x and embeddings


    return (pos_pred, heads_pred, dep_rels_pred, ner_pred)

In [11]:
import pickle

def run_and_save_posdep_model(model, logdir, bpemb_pl, le_upostag, le_deprel, params, epochs, step):

  # set trainable layers:
  layer_names = [layer.name for layer in model.layers]
  output_ner_idx = layer_names.index('output_ner')
  rnn_ner_idx = layer_names.index('rnn_ner')
  model.layers[output_ner_idx].trainable = False  
  model.layers[rnn_ner_idx].trainable = False  


  decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.9)
  optim = tf.optimizers.Adamax(params['lr'])
  global_step = 0
  history_f1 = []
  best_f1 = .0

  t0 = time.time()
  logger = logging.getLogger('tensorflow')
  logger.setLevel(logging.INFO)

  print("start training")
  
  for epoch in range(epochs):
    # TRAINING
    for (tokenids, (upostags, heads, deprels, nerents)) in dataset_UD(is_training=True, params=params, bpemb_pl=bpemb_pl, le_upostag=le_upostag, le_deprel=le_deprel):
      with tf.GradientTape() as tape:

        y_pos, y_heads, y_deprels, _ = model(tokenids, training=True)

        sent_lengths = tf.math.count_nonzero(tf.greater_equal(upostags, 0),axis=1)
        sent_lengths_weight_mask = tf.sequence_mask(sent_lengths, dtype=tf.float32)
        
        upostags_masked = tf.math.multiply(tf.dtypes.cast(sent_lengths_weight_mask,dtype=tf.int32), upostags)
        loss_upostags = tfa.seq2seq.sequence_loss(logits=y_pos, targets=upostags_masked, weights = sent_lengths_weight_mask)
        loss_upostags = tf.reduce_mean(loss_upostags)

        heads_masked = tf.math.multiply(tf.dtypes.cast(sent_lengths_weight_mask,dtype=tf.int32), heads)
        loss_heads = tf.keras.losses.sparse_categorical_crossentropy(y_pred=y_heads, y_true=heads_masked, from_logits=True)
        loss_heads = tf.reduce_mean(loss_heads)

        deprels_masked = tf.math.multiply(tf.dtypes.cast(sent_lengths_weight_mask,dtype=tf.int32), deprels)
        loss_deprels = tfa.seq2seq.sequence_loss(logits=y_deprels, targets=deprels_masked, weights = sent_lengths_weight_mask)
        loss_deprels = tf.reduce_mean(loss_deprels)

        loss = 5.0 * loss_upostags + loss_heads + loss_deprels
      
      optim.lr.assign(decay_lr(global_step))
      grads = tape.gradient(loss, model.trainable_variables)
      grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
      optim.apply_gradients(zip(grads, model.trainable_variables))

      if global_step % 50 == 0:
        logger.info("Step {} | Loss: {:.3f} | loss_upostags: {:.3f} | Loss_heads: {:.3f} | Loss_deprels: {:.3f} | Spent: {:.1f} secs | LR: {:.6f}".format(
            global_step, loss.numpy().item(), loss_upostags.numpy().item(), loss_heads.numpy().item(), loss_deprels.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
        t0 = time.time()
      global_step += 1

    # EVALUATION
    pos_true = [] ; pos_pred = []
    heads_true = [] ; heads_pred = []
    deprels_true = [] ; deprels_pred = []
    
    for (tokenids, (upostags, heads, deprels, nerents)) in dataset_UD(is_training=False, params=params, bpemb_pl=bpemb_pl, le_upostag=le_upostag, le_deprel=le_deprel):
      
      y_pos, y_heads, y_deprels, _ = model(tokenids, training=False)
      y_pos = tf.argmax(y_pos, -1)
      y_heads = tf.argmax(y_heads, -1)
      y_deprels = tf.argmax(y_deprels, -1)
      
      pos_true += upostags.numpy().flatten().tolist()
      pos_pred += y_pos.numpy().flatten().tolist()
      heads_true += heads.numpy().flatten().tolist()
      heads_pred += y_heads.numpy().flatten().tolist()
      deprels_true += deprels.numpy().flatten().tolist()
      deprels_pred += y_deprels.numpy().flatten().tolist()
      
    f1_pos   = f1_score(y_true = pos_true,
                        y_pred = pos_pred,
                        labels = list( le_upostag.classes_ ),
                        sample_weight = np.sign(pos_true),
                        average='micro',)
    history_f1.append(f1_pos)

    logger.info('\n'+classification_report(y_true = pos_true,
                                          y_pred = pos_pred,
                                          labels = list( le_upostag.transform(le_upostag.classes_) ),  
                                          target_names = list(le_upostag.classes_),
                                          digits=3))
    
    pos_accuracy = accuracy_score(y_true = pos_true, y_pred = pos_pred)
                   
    # report dependency parser's UAS:
    tp_hits = 0
    for h_pred, h_true in zip(heads_pred, heads_true):
      if h_true >= 0 and h_pred==h_true: tp_hits += 1
    uas = (float(tp_hits)/float(len(heads_true)))
    print("UAS: %.3f" % uas )
    
    # report dependency parser's LAS:
    tp_hits = 0
    for d_pred, d_true in zip(deprels_pred, deprels_true):
      if d_true >= 0 and round(d_pred)==d_true: tp_hits += 1
    print("LAS: %.3f" % (float(tp_hits)/float(len(deprels_true))) )
    
    elapsed_time = time.time() - start_time
    print(time.strftime("Elapsed: %H:%M:%S", time.gmtime(elapsed_time)))
    print("Finished epoch "+str(epoch))

    #if f1_pos > best_f1: best_f1 = f1_pos
    #  # you can save model here
    #logger.info("Best POS F1: {:.3f}".format(best_f1))
    
    #if len(history_f1) > params['num_patience'] and is_descending(history_f1):
    #  logger.info("Testing POS Accuracy not improved over {} epochs, Early Stop".format(params['num_patience']))
    #  break
    
    print('saving model')
    model.save("/content/drive/My Drive/combo/models_posdep/model_epoch_"+str(epoch))
    if epoch == 0:
      model.summary()

  return model



### ------------------ NER

In [13]:
!ls '/content/drive/My Drive/combo/ner/onefile.conll'

'/content/drive/My Drive/combo/ner/onefile.conll'


In [14]:
from sklearn import preprocessing
import os, pickle

labels = set([])
def get_label_encoders_ner(f_path):
  le_ner = preprocessing.LabelEncoder()
  
  data_file = open(f_path, "r", encoding="utf-8")
  for line in data_file:
    toks = line.split('\t')
    if len(toks)>1: labels.add( toks[1].strip() )
  print(labels)
  le_ner.fit(list(labels))
  return le_ner

le_ner = get_label_encoders_ner('/content/drive/My Drive/combo/ner/onefile.conll')
pickle.dump(le_ner, open('/content/drive/My Drive/combo/ner/le_ner.pkl', 'wb'))

{'U-geogName', 'U-date', 'I-geogName', 'B-persName', 'L-persName', 'L-geogName', 'O', 'U-time', 'L-orgName', 'B-geogName', 'B-placeName', 'U-orgName', '-', 'L-placeName', 'B-date', 'U-persName', 'I-orgName', 'B-orgName', 'I-persName', 'U-placeName', 'L-date'}


In [15]:
# this method iterates CONLL files and outputs padded IDs  
# it's intended as TensorFlow graph input
def data_generator_ids_NER(f_path, bpemb_pl, le_ner): #, params, bpemb_pl):
  orths = [] ; namedents = []        
  print('Reading', f_path)
  data_file = open(f_path, "r", encoding="utf-8")
  
  for line in data_file:
    toks = line.split('\t')
    if len(toks)>1: 
      orths.append( toks[0] )
      namedents.append( toks[1].strip() )
    else:  
      # subword conversion:
      sub_tokenids = []
      sub_namedents = []        
      for nr, orth in enumerate(orths):
        tokenids = bpemb_pl.encode_ids(orth)
        sub_tokenids.extend( tokenids )
        sub_namedents.extend( [namedents[nr]] * len(tokenids) )
        
      # transform labels:
      sub_namedents_ids = le_ner.transform(sub_namedents)
      free_slot = [0] * len(sub_tokenids)

      orths = [] ; namedents = [] 

      yield (sub_tokenids, (free_slot, free_slot, free_slot, sub_namedents_ids ))


In [ ]:
data_generator_ids_NER('/content/drive/My Drive/combo/ner/onefile.conll', bpemb_pl, le_ner)

In [17]:
def dataset_NER(is_training, params, bpemb_pl, le_ner):

  _shapes = ( (None,), ((None,), (None,), (None,), (None,)))
  _types = (tf.int32, (tf.int32, tf.int32, tf.int32, tf.int32))
  _pads = (0, (0, 0, 0, 0)) # no neg values here!
  
  if is_training:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator_ids_NER(params['train_path_ner'], bpemb_pl, le_ner),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.shuffle(params['num_samples'])
    ds = ds.padded_batch(params['batch_size'], _shapes )#, _pads) # default mask is 0
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  else:
    ds = tf.data.Dataset.from_generator(
      lambda: data_generator_ids_NER(params['test_path_ner'], bpemb_pl, le_ner),
      output_shapes = _shapes,
      output_types = _types,)
    ds = ds.padded_batch(params['batch_size'], _shapes )#, _pads) # default mask is 0
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
  
  return ds

In [18]:
import pickle

def run_and_save_ner_model(model, logdir, params, bpemb_pl, le_ner, epochs, step):

  # set only ner layers to be trainable:
  layer_names = [layer.name for layer in model.layers]
  rnn_ner_idx = layer_names.index('rnn_ner')
  output_ner_idx = layer_names.index('output_ner')
  for layer in model.layers:
    layer.trainable=False 
  model.layers[rnn_ner_idx].trainable = True
  model.layers[output_ner_idx].trainable = True


  decay_lr = tf.optimizers.schedules.ExponentialDecay(params['lr'], 1000, 0.9)
  optim = tf.optimizers.Adamax(params['lr'])
  global_step = 0
  history_f1 = []
  best_f1 = .0

  t0 = time.time()
  logger = logging.getLogger('tensorflow')
  logger.setLevel(logging.INFO)

  print("start NER training")
  
  for epoch in range(epochs):
    # TRAINING
    for (tokenids, (upostags, heads, deprels, nerents)) in dataset_NER(is_training=True, params=params, bpemb_pl=bpemb_pl, le_ner=le_ner):
      with tf.GradientTape() as tape:

        _, _, _, y_namedents = model(tokenids, training=True)

        sent_lengths = tf.math.count_nonzero(tf.greater_equal(nerents, 0),axis=1)
        sent_lengths_weight_mask = tf.sequence_mask(sent_lengths, dtype=tf.float32)
        
        ner_masked = tf.math.multiply(tf.dtypes.cast(sent_lengths_weight_mask,dtype=tf.int32), nerents)
        loss_ner = tfa.seq2seq.sequence_loss(logits=y_namedents, targets=ner_masked, weights = sent_lengths_weight_mask)
        loss = tf.reduce_mean(loss_ner)
      
      optim.lr.assign(decay_lr(global_step))
      grads = tape.gradient(loss, model.trainable_variables)
      grads, _ = tf.clip_by_global_norm(grads, params['clip_norm'])
      optim.apply_gradients(zip(grads, model.trainable_variables))

      if global_step % 50 == 0:
        logger.info("Step {} | Loss: {:.5f} | Spent: {:.1f} secs | LR: {:.6f}".format(
            global_step, loss.numpy().item(), time.time()-t0, optim.lr.numpy().item()))
        t0 = time.time()
      global_step += 1

    # EVALUATION
    ner_true = [] ; ner_pred = []
    
    for (tokenids, (upostags, heads, deprels, nerents)) in dataset_NER(is_training=False, params=params, bpemb_pl=bpemb_pl, le_ner=le_ner):
      
      _, _, _, y_namedents = model(tokenids, training=False)
      y_ner = tf.argmax(y_namedents, -1)
      
      ner_true += nerents.numpy().flatten().tolist()
      ner_pred += y_ner.numpy().flatten().tolist()
      
    f1_ner   = f1_score(y_true = ner_true,
                        y_pred = ner_pred,
                        labels = list( le_ner.classes_ ),
                        sample_weight = np.sign(ner_true),
                        average='micro',)
    history_f1.append(f1_ner)

    logger.info('\n'+classification_report(y_true = ner_true,
                                          y_pred = ner_pred,
                                          labels = list( le_ner.transform(le_ner.classes_) ),  
                                          target_names = list(le_ner.classes_),
                                          digits=3))
    
    ner_accuracy = accuracy_score(y_true = ner_true, y_pred = ner_pred)
    
    elapsed_time = time.time() - start_time
    print(time.strftime("Elapsed: %H:%M:%S", time.gmtime(elapsed_time)))
    print("Finished epoch "+str(epoch))

    
    print('saving model')
    model.save("/content/drive/My Drive/combo/models_posdep_ner/model_epoch_"+str(epoch))
    if epoch == 0:
      model.summary()

  return model


In [20]:
epochs_posdep = 45
epochs_ner = 20

bpemb_pl = BPEmb(lang="pl", dim=params['embedding_size'], vs=params['vocabulary_size'])
embedding_matrix = bpemb_pl.vectors

le_upostag, le_deprel = get_label_encoders(bpemb_pl)
le_ner = pickle.load(open('/content/drive/My Drive/combo/ner/le_ner.pkl', 'rb'))

start_time = time.time()

model = Model(params, embedding_matrix, bpemb_pl, le_upostag, le_deprel, le_ner)
model.build(input_shape=(None ,params["sentence_len"],)) # None is batch

# def run_and_save_model(model, logdir, bpemb_pl, le_upostag, le_deprel, params, epochs, step):
model = run_and_save_posdep_model(model, 'logs/final_model/' , bpemb_pl, le_upostag, le_deprel, params, epochs_posdep, 0)


# def run_and_save_model(logdir, params, epochs, step):
model = run_and_save_ner_model(model, 'logs/final_model/', params, bpemb_pl, le_ner, epochs_ner, 0)

model.save("1model_final")


downloading https://nlp.h-its.org/bpemb/pl/pl.wiki.bpe.vs50000.model


100%|██████████| 1135152/1135152 [00:00<00:00, 1636995.72B/s]


downloading https://nlp.h-its.org/bpemb/pl/pl.wiki.bpe.vs50000.d100.w2v.bin.tar.gz


100%|██████████| 19000997/19000997 [00:02<00:00, 8861930.10B/s]


start training
Reading /content/drive/My Drive/combo/PTCombo/UD/PL/pl_pdb-ud-train.conllu
INFO:tensorflow:Step 0 | Loss: 21.535 | loss_upostags: 2.905 | Loss_heads: 3.455 | Loss_deprels: 3.556 | Spent: 7.5 secs | LR: 0.000300
INFO:tensorflow:Step 50 | Loss: 20.743 | loss_upostags: 2.735 | Loss_heads: 3.683 | Loss_deprels: 3.386 | Spent: 7.5 secs | LR: 0.000298
INFO:tensorflow:Step 100 | Loss: 19.551 | loss_upostags: 2.593 | Loss_heads: 3.990 | Loss_deprels: 2.597 | Spent: 7.7 secs | LR: 0.000297
INFO:tensorflow:Step 150 | Loss: 19.083 | loss_upostags: 2.616 | Loss_heads: 3.769 | Loss_deprels: 2.237 | Spent: 6.9 secs | LR: 0.000295
INFO:tensorflow:Step 200 | Loss: 18.318 | loss_upostags: 2.530 | Loss_heads: 3.802 | Loss_deprels: 1.868 | Spent: 8.1 secs | LR: 0.000294
INFO:tensorflow:Step 250 | Loss: 17.978 | loss_upostags: 2.476 | Loss_heads: 3.804 | Loss_deprels: 1.795 | Spent: 7.4 secs | LR: 0.000292
INFO:tensorflow:Step 300 | Loss: 18.621 | loss_upostags: 2.548 | Loss_heads: 4.154 | 

...

Elapsed: 04:54:25
Finished epoch 19
saving model


INFO:tensorflow:Assets written to: /content/drive/My Drive/combo/models_posdep_ner/model_epoch_19/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/combo/models_posdep_ner/model_epoch_19/assets


INFO:tensorflow:Assets written to: 1model_final/assets


INFO:tensorflow:Assets written to: 1model_final/assets


In [23]:
!ls '/content/drive/My Drive/combo/model_final'

model_final


In [25]:
!mv 1model_final '/content/drive/My Drive/combo/model_final'

In [36]:
!ls -lh '/content/drive/My Drive/combo/model_final/1model_final'

total 217M
drwx------+ 2 root root 4.0K Jun 22 10:29 assets
-rw-------+ 1 root root 100M Jun 22 10:29 keras_metadata.pb
-rw-------+ 1 root root 117M Jun 22 10:29 saved_model.pb
drwx------+ 2 root root 4.0K Jun 22 10:29 variables


In [41]:
!ls '/content/drive/My Drive/combo/model_final/'

1model_final  model_final


In [42]:
!zip -r model_final.zip '/content/drive/My Drive/combo/model_final/1model_final/'

  adding: content/drive/My Drive/combo/model_final/1model_final/ (stored 0%)
  adding: content/drive/My Drive/combo/model_final/1model_final/variables/ (stored 0%)
  adding: content/drive/My Drive/combo/model_final/1model_final/variables/variables.data-00000-of-00001 (deflated 7%)
  adding: content/drive/My Drive/combo/model_final/1model_final/variables/variables.index (deflated 60%)
  adding: content/drive/My Drive/combo/model_final/1model_final/assets/ (stored 0%)
  adding: content/drive/My Drive/combo/model_final/1model_final/saved_model.pb (deflated 62%)
  adding: content/drive/My Drive/combo/model_final/1model_final/keras_metadata.pb (deflated 57%)


In [43]:
!ls

drive  model_final.zip	sample_data
